# Installations

In [ ]:
#!apt-get update
#!apt-get install cuda-10.1
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
#!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-810qg1i0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-810qg1i0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=9a29b7b10d20f6c7174d70076474275e9d8e0d2272e50e8dbe66e970508bfbc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-3y4_5r44/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


# Lab 3

In [ ]:
lab3 = """
// Zhazira Kabdrakhmetova || ID: 201980168
// Course: CSCI325 Fall23 || LA3

#define N (512*512)
// My system wasn't able to run code with 1024x1024, so I decreased it
// it was due to my hardware or google colab limitations
#define K 4
#define K2 32
#define K3 512
#define K4 256
#include <stdio.h>
#include <time.h>

// CONCLUSION: WORST CONFIG WAS 1 BLOCK USAGE
// BEST CONFIG WAS USING 512 THREADS PER BLOCK, 512 BLOCKS
// WITH INCREASING NUMBER OF THREADS PER BLOCK RESULTS WERE IMPROVED

// ALSO WITH MORE THREADS IN A SINGLE BLOCK PERFORMANCE IMPROVES
// MAYBE BECAUSE IF THERE ARE TOO MANY BLOCKS WE WILL HAVE A LOT OF
// ATOMIC FUNCTIONS AND BLOCK COORDINATION
// IT'S BETTER TO SPREAD THE TASKS EVENLY

// BUT HAVING JUST ONE BLOCK WITH SAME NUMBER OF THREADS(512)
// IS NOT EFFICIENT - BAD MEMORY UTILIZATION, SO SLOWER EXEC TIME

void dpSequential(float A[N], float B[N], float *c) {
    for (int i = 0; i < N; i++) {
        *c = *c + A[i] * B[i];
    }
}

__global__ void dpGlobal(float A[N], float B[N], float *c){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    atomicAdd(c, new_c);
}

__global__ void dpShared_Conf1(float A[N], float B[N], float *c){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    // start a variable to store all c values accross threads
    __shared__ float new_cs[K];
    new_cs[threadIndex] = new_c;
    __syncthreads();

    // parallel reduction to find a sum of separate c values accross threads
    // after the iteration decrease the scope by two
    // continually add array elements until the last c values is new_cs[0]
    for (int i = threadNum / 2; i > 0; i /= 2) {
        if (threadIndex < i) {
            new_cs[threadIndex] += new_cs[threadIndex + i];
        }
    }
     __syncthreads();

    // make this equality only once
    if (threadIndex == 0) {
        *c = new_cs[0];
    }
}

__global__ void dpShared_ConfMany(float A[N], float B[N], float *new_cs){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    __shared__ float new_block_cs[K];
    new_block_cs[threadIdx.x] = new_c;
    __syncthreads();

    for (int i = blockDim.x / 2; i > 0; i /= 2) {
        if (threadIdx.x < i) {
            new_block_cs[threadIdx.x] += new_block_cs[threadIdx.x + i];
        }
        __syncthreads();
    }
    if (threadIdx.x == 0) {
        new_cs[blockIdx.x] = new_block_cs[0];
    }
}

__global__ void dpShared_ConfMany2(float A[N], float B[N], float *new_cs){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    __shared__ float new_block_cs[K2];
    new_block_cs[threadIdx.x] = new_c;
    __syncthreads();

    for (int i = blockDim.x / 2; i > 0; i /= 2) {
        if (threadIdx.x < i) {
            new_block_cs[threadIdx.x] += new_block_cs[threadIdx.x + i];
        }
        __syncthreads();
    }
    if (threadIdx.x == 0) {
        new_cs[blockIdx.x] = new_block_cs[0];
    }
}

__global__ void dpShared_ConfMany3(float A[N], float B[N], float *new_cs){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    __shared__ float new_block_cs[K3];
    new_block_cs[threadIdx.x] = new_c;
    __syncthreads();

    for (int i = blockDim.x / 2; i > 0; i /= 2) {
        if (threadIdx.x < i) {
            new_block_cs[threadIdx.x] += new_block_cs[threadIdx.x + i];
        }
        __syncthreads();
    }
    if (threadIdx.x == 0) {
        new_cs[blockIdx.x] = new_block_cs[0];
    }
}

__global__ void dpShared_ConfMany4(float A[N], float B[N], float *new_cs){
    int threadIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int threadNum = blockDim.x * gridDim.x;
    float new_c = 0;

    for (int i = threadIndex; i < N; i += threadNum) {
        new_c = new_c + A[i] * B[i];
    }

    __shared__ float new_block_cs[K4];
    new_block_cs[threadIdx.x] = new_c;
    __syncthreads();

    for (int i = blockDim.x / 2; i > 0; i /= 2) {
        if (threadIdx.x < i) {
            new_block_cs[threadIdx.x] += new_block_cs[threadIdx.x + i];
        }
        __syncthreads();
    }
    if (threadIdx.x == 0) {
        new_cs[blockIdx.x] = new_block_cs[0];
    }
}

// TO DO: create more kernel function names.

int main(){
    clock_t start_time, end_time;

    float A[N], B[N];
    float c=0;

    for (int i = 0; i < N; i++){
        A[i] = 1.0f;
        B[i] = 2.0f;
    }

    start_time = clock();
    // FIRST: SEQUENTIAL
    dpSequential(A, B, &c);
    printf("result of sequential = %.2f\\n", c);
    end_time = clock();
    double elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 1: %f \\n", elapsed_time);




    // CUDA MEMORY MANAGEMENT
    float *deviceA, *deviceB, *deviceC;
    cudaMalloc((void **)&deviceA, N * sizeof(float));
    cudaMalloc((void **)&deviceB, N * sizeof(float));
    cudaMalloc((void **)&deviceC, sizeof(float));

    cudaMemcpy(deviceA, A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, B, N * sizeof(float), cudaMemcpyHostToDevice);


    start_time = clock();
    // SECOND: GLOBAL MEMORY
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    dpGlobal<<<(N + K - 1) / K, K>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    printf("result of Global memory = %.2f\\n", c);
    end_time = clock();
    elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 2: %f \\n", elapsed_time);


    start_time = clock();
    // THIRD: SHARED MEMORY - 1 BLOCK ONLY
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(32,32);
    dpShared_Conf1<<<1, threadsPerBlock>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    printf("result of Shared memory (1 block) = %.2f\\n", c);
    end_time = clock();
     elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 3: %f \\n", elapsed_time);


    start_time = clock();
    // FOURTH: SHARED MEMORY - MULTIPLE BLOCKS
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    int numBlocks = (N + K - 1) / K;
    dpShared_ConfMany<<<numBlocks, K>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    float result = 0.0f;

    for (int i = 0; i < numBlocks; i++) {
        result += c;
    }
    printf("result of Shared memory (multiple blocks K=4) = %.2f\\n", result);
    end_time = clock();
     elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 4: %f \\n", elapsed_time);


    start_time = clock();
    // FIFTH: SHARED MEMORY - MULTIPLE BLOCKS K = 32
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    numBlocks = (N + K2 - 1) / K2;
    dpShared_ConfMany2<<<numBlocks, K2>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    result = 0.0f;

    for (int i = 0; i < numBlocks; i++) {
        result += c;
    }
    printf("result of Shared memory (multiple blocks K=32) = %.2f\\n", result);
    end_time = clock();
     elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 5: %f \\n", elapsed_time);


    start_time = clock();
    // SIXTH: SHARED MEMORY - MULTIPLE BLOCKS K = 256
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    numBlocks = (N + K4 - 1) / K4;
    dpShared_ConfMany3<<<numBlocks, K4>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    result = 0.0f;

    for (int i = 0; i < numBlocks; i++) {
        result += c;
    }
    printf("result of Shared memory (multiple blocks K=256) = %.2f\\n", result);
    end_time = clock();
     elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 6: %f \\n", elapsed_time);


    start_time = clock();
    // SEVENTH: SHARED MEMORY - MULTIPLE BLOCKS K = 512
    c=0;
    cudaMemcpy(deviceC, &c, sizeof(float), cudaMemcpyHostToDevice);

    numBlocks = (N + K3 - 1) / K3;
    dpShared_ConfMany3<<<numBlocks, K3>>>(deviceA, deviceB, deviceC);
    cudaMemcpy(&c, deviceC, sizeof(float), cudaMemcpyDeviceToHost);
    result = 0.0f;

    for (int i = 0; i < numBlocks; i++) {
        result += c;
    }
    printf("result of Shared memory (multiple blocks K=512) = %.2f\\n", result);
    end_time = clock();
        elapsed_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("Elapsed time for config 7: %f \\n", elapsed_time);

    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    return 0;


  // TO DO
  //1. Invoke the kernel with CUDA memory management
  //2. Verify the result of the scholar value (MUST be 2*N = 2*1024*1024 = 2097152)
}

"""

In [ ]:
text_file = open("Zhazira_Kabdrakhmetova_LA3.cu", "w")
text_file.write(lab3)
text_file.close()
!nvcc -o Zhazira_Kabdrakhmetova_LA3 Zhazira_Kabdrakhmetova_LA3.cu -Xcompiler -fopenmp -lgomp
!./Zhazira_Kabdrakhmetova_LA3

result of sequential = 524288.00
Elapsed time for config 1: 0.001051 
result of Global memory = 524288.00
Elapsed time for config 2: 0.001081 
result of Shared memory (1 block) = 3751936.00
Elapsed time for config 3: 0.002136 
result of Shared memory (multiple blocks K=4) = 524288.00
Elapsed time for config 4: 0.000543 
result of Shared memory (multiple blocks K=32) = 524288.00
Elapsed time for config 5: 0.000106 
result of Shared memory (multiple blocks K=256) = 524288.00
Elapsed time for config 6: 0.000065 
result of Shared memory (multiple blocks K=512) = 524288.00
Elapsed time for config 7: 0.000065 
